## The N-Body Problem
This is a classic problem that's been of interest since the time of Newton. As it relates to the positions in of heavenly bodies, it was at the center of great debates that took place during the 1500s and involved such greats as Tycho Brahe (my favorite), Nicolaus Copernicus, Galileo, and Kepler.

We'll attack the problem with a brute force approach, adding up all the forces occurring between bodies. However, many good alternatives exist, and will be the topic of at least one graduate project.

### Two bodies
Consider a force such as gravity, which pulls the moon toward the Earth *and the Earth toward the moon*. Generalize these two bodies to a pair of particles.

Suppose the particles are numbered 1 and 2. The force on 1 due to 2 will be labeled $\mathbf{F}_{12}$. Note it is a vector quantity. What about the force on 2 due to 1? Well, it's going to be anti-symmetric in the cases we will consider, $\mathbf{F}_{12} = -\mathbf{F}_{21}$. What might this force look like? It will be a *central force*, expressed as

$$\mathbf{F}_{12}(\mathbf{r}) = |F_{12}(r)|\mathbf{\hat r}_{12}.$$

This is like what we've seen before where the magnitude is abstracted as $ |\mathbf{F}_{12}(\mathbf{r})|$, and we focus on the direction in $\mathbf{\hat{r}}_{12}$. 

This direction is given by
$$\mathbf{\hat{r}}_{12} = \frac{\mathbf{r}_1 - \mathbf{r}_2}{|\mathbf{r}_1 - \mathbf{r}_2|},$$
which points from 2 to 1.

![The difference in vectors.](vector_diff.png)

So, now we know the direction of this abstract force. How do we account for its working in both directions? That is particle 1 on 2 and 2 on 1.

It is convenient to write the forces in a matrix, where the matrix entry $\mathcal{F}_{ij}$ is the force of $j$ on $i$. There is no self-interaction; $\mathcal{F}_{ii}$=0.

$$\mathcal{F} =
\begin{bmatrix}
0 & \mathbf{F}_{12}  \\
\mathbf{F}_{21}&0\\
\end{bmatrix}
=
\begin{bmatrix}
0 & \mathbf{F}_{12}  \\
-\mathbf{F}_{12}&0\\
\end{bmatrix}$$

Such a matrix has two important properties; 
* the sum of each row is the total force on particle $i$ - $\mathbf{F}_i = \sum_j \mathcal{F}_{ij}$
* the matrix is anti-symmetric, so only the upper triangle needs to be computed.

But, what about those entries in that matrix - it looks like **they should be vectors**. In practice, it will be useful to maintain a scalar matrix for each dimension, a matrix of the $x$ components of force, $\mathcal{F}^x_{ij}$, and $y$ components, $\mathcal{F}^y_{ij}$, etc. 

### Three and more bodies
It probably occurred to you that the matrices are a lot of work when the forces are just $\mathbf{F}_{12}$ and $-\mathbf{F}_{12}$; the result of a single calculation. But, watch what happens with three bodies.

![The positions of and vectors between three bodies.](3body.png)

$$\mathcal{F} =
\begin{bmatrix}
0 & \mathbf{F}_{12} & \mathbf{F}_{13} \\
\mathbf{F}_{21} & 0 & \mathbf{F}_{23}\\
\mathbf{F}_{31}  & \mathbf{F}_{32} & 0
\end{bmatrix}$$

In this case, the sums of the rows are critical because because each body is attracted to 2 other bodies. But the formalism is powerful and appropriate. How many unique calculations need to be done? 

$$\mathcal{F} =
\begin{bmatrix}
0 & \mathbf{F}_{12} & \mathbf{F}_{13} \\
-\mathbf{F}_{12} & 0 & \mathbf{F}_{23}\\
-\mathbf{F}_{13}  & -\mathbf{F}_{23} & 0
\end{bmatrix}$$

Just 3, the upper triangle of the matrix. The most important aspect is, again, the fact that the total force on each particle is computed by summing its row:

$$\mathbf{F}_i = \sum_j \mathcal{F}_{ij}$$

With this statement, I think we see the sketch of the n-body algorithm. First we fill in a matrix:

`for i in range(len(particles):
    for j range(i+1,len(particles)):
        rij = particles.r(i) - particles.r(j)
        Fmatrix[i,j] = f(rij) * rij / norm(rij)
`

Then we find the force on each particle with sums over the rows of `FMatrix`

The generalization from three bodies to n-bodies is superficial.

There are opportunities to vectorize the operations by using `repmat` to create highly redundant arrays of coordinate data. Doing this may interest you, but it isn't necessary. It does tend to obfuscate code. I do wonder, but haven't tried to see if `numba` doesn't get the speedups sought.

### The force
It has been worthwhile to abstract the force. As you can see in the psuedo-code/Python above, a function handle is a nice mechanism for this. However, we'll be concerned with gravitational forces in this class. That force is

$$\mathbf{F}_{12} = -G \frac{m_1m_2}{r_{12}^2}\mathbf{\hat{r}}_{12}$$

Which can more conveniently be written

$$\mathbf{F}_{12} = -G \frac{m_1m_2}{r_{12}^3}\mathbf{r}_{12}$$

If you are curious about why we generalize, let me provide at least one example. There is a rich literature in molecular dynamics which involves the use of Lennard-Jones type potentials

$$4\epsilon \left[\left(\frac{\sigma}{r}\right)^{12}-\left(\frac{\sigma}{r}\right)^{6}\right]$$

Which describe weak attraction/strong repulsion. Forces are found by differentiation of the potential.

Of late, this concept has been extended to something called *discrete-element modeling*, which is good for large assemblages of granular material.

### Objective

Write an ODE function that provides the derivatives for the n-body problem. Assume as you have in the past that the state is entering as some vector `y` and the function will return `dydt`, or $\frac{dy}{dt}$. To match signatures, the function will have to accept a `t` or time that is not used, and a `p` which is a parameter. Here, it suggest `p` be made a dictionary of parameters.

The difficulties to be resolved in implementation include

* accounting for the masses of the particles.
* accounting for the dimension of the problem.
* accounting for the half way point, or where the position information stops, and the velocity information begins.
* not computing the same thing twice; only find $\mathbf{F}_{ij}$, not $\mathbf{F}_{ji}$. But, account for both.
* summing the columns so that each row's sum is the force on a particle.
* determining the acceleration and not the mass.
* handling a parameter called `fix_first` that prevents the first particle from moving. This is useful for simulating massive objects like the sun, and will help with a simulation of the helium atom.

Below, you will find the signature for the function, and some test data results to determine if the function is working correctly. Finish the function and verify the test data is giving the correct outputs.

In [15]:
import numpy as np
from prettytable import PrettyTable
from ode_solver import ODESolver


def n_body(t, y, p):
    """
    Write what goes in here!
    Instructions above.

    t is our time vector
    y is the state vector of all the bodies:
    [x1, y1, z1, x2, y2, z2, vx1, vy1, vz1, etc...]
    p is the object with our data

    note:

    returns dxdt
    """
    N = len(p['mass'])
    print(N)
    d = p['dimension']
    Fmatrix = np.zeros([N, N, d])

    # split y into position and velocity vectors, go from flattened pos. vector to size Nxd array
    half = y.size // 2
    pos_vec = y[:half]
    vel_vec = y[half:]
    pos_matrix = pos_vec.reshape(N, d)

    # Loop over the top right corner of the force matrix
    for i in range(0, N - 1):
        for j in range(i + 1, N):
            # Find the vector from body j to body i
            rij = pos_matrix[i, :] - pos_matrix[j, :]

            # Compute the force from body j onto body i
            Fij = p['force'](rij, p['mass'][j], p['G']) * rij

            # Fill in the symmetric pieces of the force matrix
            Fmatrix[i, j, :] = Fij
            Fmatrix[j, i, :] = -Fij

    # Compute the force vectors acting on each body
    forces = np.sum(Fmatrix, axis=1)

    # flatten the forces matrix and combine it with the vector list
    acc_vec = forces.flatten().tolist()
    dxdt = vel_vec + acc_vec

    return dxdt

def force_function(vec, m, G):
    """ Computes the force from the n-body problem """
    f = -G * m / np.power(np.linalg.norm(vec), 3)
    return f

euler      = np.array([0,0,1,0,-1,0,0,0,0,.8,0,-.8])
p = {'mass':[1,1,1],'G':1,'dimension':2, 'force':force_function,'fix_first':False}
t_vec = n_body(0, euler, p)

headings = ['RUN','x1','y1','x2','y2','x3','y3','vx1','vy1','vx2','vy2','vx3','vy3']
t = PrettyTable(headings)
t.add_row(['euler']+list(n_body(0,euler,p)))

3
3


Exception: Row has incorrect number of values, (actual) 7!=13 (expected)

In [6]:
"""
Test data - make sure that your function reproduces the following.
Why not add a row of differences and make sure they are zero?
"""

# Order is all positions then all velocities grouped by body.
# eg, three bodies in two dimensions:
# x1,y1,x2,y2,x3,y3,vx1,vy1,vx2,vy2,vx3,vy3

from prettytable import PrettyTable


euler      = np.array([0,0,1,0,-1,0,0,0,0,.8,0,-.8])
four_body  = np.array([1.382857,0,\
                   0,0.157030,\
                  -1.382857,0,\
                   0,-0.157030,\
                   0,0.584873,\
                   1.871935,0,\
                   0,-0.584873,\
                  -1.871935,0],dtype=np.float128)
helium_1 = np.array([0,0,2,0,-1,0,0,0,0,.95,0,-1])

# The data structure holding parameters. You need not
# do it this way, but it's nice.

p = {'m':[1,1,1],'G':1,'dimension':2,'fix_first':False}
p4 = {'m':[1,1,1,1],'G':1,'dimension':2,'fix_first':False}
phe = {'m':[2,-1,-1],'G':1,'dimension':2,'fix_first':True}

headings = ['RUN','x1','y1','x2','y2','x3','y3','vx1','vy1','vx2','vy2','vx3','vy3']
t = PrettyTable(headings)
t.add_row(['euler']+list(n_body(0,euler,p)))

t.add_row(['He']+list(n_body(0,helium_1,phe)))
print(t)

headings = ['RUN','x1','y1','x2','y2','x3','y3','x4','y4','vx1','vy1','vx2','vy2','vx3','vy3','vx4','vy4']
t = PrettyTable(headings)
t.add_row(['4 body']+list(n_body(0,four_body,p4)))
print(t)

KeyError: 'num_bodies'

3


Exception: Row has incorrect number of values, (actual) 7!=13 (expected)